Problem 1

In [ ]:
!pip install transformers torch fastapi uvicorn pyngrok requests beautifulsoup4


In [ ]:
# Import necessary libraries
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
import uvicorn
import threading
from pyngrok import ngrok
import socket

# Function to verify if a port is in use
def is_port_in_use(port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

port = 8002
if is_port_in_use(port):
    print(f"Port {port} is already in use. Try stopping conflicting processes or choose another port.")
else:
    # Initialize the Hugging Face QA model pipeline
    qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")
    def answer_question(context, question):
        result = qa_pipeline({"context": context, "question": question})
        return result.get("answer", "I don't know the answer")

    # Initialize the FastAPI application
    app = FastAPI()

    # Define the data model for incoming requests
    class QARequest(BaseModel):
        url: str
        question: str

    # Endpoint to answer questions based on webpage content
    @app.post('/question_answering')
    def answer_question_from_web(data: QARequest):
        # Fetch webpage content
        try:
            response = requests.get(data.url)
            response.raise_for_status()
        except requests.RequestException as e:
            raise HTTPException(status_code=400, detail=str(e))

        # Parse the content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        page_content = soup.get_text(separator=' ')

        # Answer the question using the Hugging Face pipeline
        result = answer_question(page_content, data.question)
        if result == "I don't know the answer" or not result:
            return {"answer": "I don’t know the answer"}
        return {"answer": result}

    # Function to run the Uvicorn server on the selected port
    def start_uvicorn():
        uvicorn.run(app, host='0.0.0.0', port=port)

    # Start the server on a separate thread
    server_thread = threading.Thread(target=start_uvicorn, daemon=True)
    server_thread.start()

    # Set up ngrok authtoken
    ngrok.set_auth_token("2g2gInvCiLEokbX8kQl0H9o2OoQ_3ixAnVMYPRC5CJ76qEmHa")

    # Establish a tunnel using HTTP protocol with the chosen port
    public_tunnel = ngrok.connect(addr=str(port), proto="http")
    public_url = public_tunnel.public_url
    print(f'Public URL: {public_url}')

    # Example input data
    data = {
        "url": "https://en.wikipedia.org/wiki/Generative_artificial_intelligence",
        "question": "What are the concerns around Generative AI?"
    }

    # Make the POST request to the FastAPI server
    response = requests.post(f'{public_url}/question_answering', json=data)
    print(response.json())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO:     Started server process [45587]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8002 (Press CTRL+C to quit)


Public URL: https://aa4b-34-145-55-29.ngrok-free.app
INFO:     34.145.55.29:0 - "POST /question_answering HTTP/1.1" 200 OK
{'answer': 'Cybersecurity and Privacy'}


Problem 2

In [2]:
pip install fastapi uvicorn transformers torch


In [4]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import pipeline
import uvicorn
import threading
from pyngrok import ngrok

# Initialize the FastAPI application
app = FastAPI()

# Load the GPT-2 text generation pipeline
generator = pipeline("text-generation", model="gpt2")
class ContentRequest(BaseModel):
    topic: str
    format: str

# Function to generate text based on format
def generate_text(topic: str, format: str) -> str:
    # Define templates for each format with specific instructions
    templates = {
        'linkedin post': (
            f"Write an engaging LinkedIn post about {topic}. "
            f"Explain why this topic is important in today's world, highlight key benefits, and encourage discussion using hashtags like #Innovation and #TechRevolution. "
            f"Include emojis: 🚀🤖💡."
        ),
        'email': (
            f"Write an informative email about {topic}, providing valuable insights and useful data. "
            f"Include practical tips and references for further reading."
        ),
        'blog': (
            f"Write a blog post about {topic}, focusing on practical applications, challenges, and future prospects. "
            f"Make sure to cover key aspects and add relevant examples."
        )
    }

    # Check if the requested format is available
    prompt = templates.get(format.lower())
    if not prompt:
        raise ValueError("Unsupported format. Please use 'linkedin post', 'email', or 'blog'.")

    # Generate the text using the GPT-2 pipeline
    response = generator(
        prompt,
        max_length=150,  # Adjust as per your requirement
        num_return_sequences=1,
        temperature=0.7,       # Adjust temperature
        top_p=0.9,             # Nucleus sampling to improve relevance
        truncation=True,       # Ensure truncation is active
    )
    return response[0]["generated_text"].strip()

# Define content generation endpoint
@app.post('/generate_content')
def generate_content(data: ContentRequest):
    try:
        result = generate_text(data.topic, data.format)
    except ValueError as e:
        raise HTTPException(status_code=400, detail=str(e))
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error generating content: {str(e)}")

    # Return the generated content
    return {"text": result}

# Function to run the Uvicorn server
def start_uvicorn():
    uvicorn.run(app, host='0.0.0.0', port=8004)

# Start the server on a separate thread
server_thread = threading.Thread(target=start_uvicorn, daemon=True)
server_thread.start()

# Set up ngrok authtoken
ngrok.set_auth_token("2g2gInvCiLEokbX8kQl0H9o2OoQ_3ixAnVMYPRC5CJ76qEmHa")

# Establish a tunnel using HTTP protocol with the selected port
public_tunnel = ngrok.connect(addr="8004", proto="http")
public_url = public_tunnel.public_url
print(f'Public URL: {public_url}')

# Example input data for testing
import requests

data = {
    "format": "linkedin post",
    "topic": "Generative AI"
}

# Make the POST request to the FastAPI server
response = requests.post(f'{public_url}/generate_content', json=data)
print(response.json())


INFO:     Started server process [740]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8004): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


Public URL: https://9493-34-122-193-123.ngrok-free.app


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


INFO:     34.122.193.123:0 - "POST /generate_content HTTP/1.1" 200 OK
{'text': "Write an engaging LinkedIn post about Generative AI. Explain why this topic is important in today's world, highlight key benefits, and encourage discussion using hashtags like #Innovation and #TechRevolution. Include emojis: 🚀🤖💡.\n\n🤖💡. Make sure you're following our team and have your own personal Twitter account 🤖💡.\n\nWhat are the benefits of Generative AI?\n\nIt allows you to create the most effective workflows and execute the most effective tasks, but it also makes it easier to learn and perform. It's also a great way to learn about AI, which is why it's a great way to start learning about"}


More powerful model instead of GPT-2, such as GPT-2 medium